### 几种特征提取技术：
* 词袋模型
* TF-IDF模型
* 高级词向量模型

#### 词袋模型

词袋模型也许是从文本文档中提取特征最简单但又最有效的技术。这个模型的本质时将文本文档转化为向量，从而将每个文档转化为一个向量，这个向量表示在文档空间中全部不同的单词在该文档中出现的频率。因此，根据前面的数学定义，这里的例子向量记为D，每个单词的权重与该词在文档中出现的频率相等。
可以为单个单词出现频率和n元分词出现频率建立同样的模型，该模型就是n元分词词袋模型，他计算不同的n元分词在每个文档中出现的频率。
下面的代码片段给出了一个函数，实现了基于词袋的特征提取模块，该模块也接受ngram_range参数作为n元分词的特征。

In [1]:
CORPUS = [
    'the sky is blue',
    'sky is blue and sky is beautiful',
    'the beautiful sky is so blue',
    'i love blue cheese'
]
new_doc = ['loving this blue sky today']

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
def bow_extractor(corpus,ngram_range = (1,3)):
    vectorizer = CountVectorizer(min_df = 1,ngram_range = ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer,features

上面的函数使用CountVectorizer类，可以设置ngram_range为不同的参数值，如(1,3)，将建立包括所有unigram、bigram和trigram的向量空间。下面的代码片段显示函数在样本语料，即4个训练文档和1个测试文档中的执行情况。

In [3]:
#import nltk
bow_vectorizer,bow_features = bow_extractor(CORPUS)
features = bow_features.todense()
print (features)

[[0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1]
 [1 1 1 1 0 0 1 1 1 0 0 2 1 1 1 0 0 0 0 0 2 2 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0]
 [0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [4]:
new_doc_features = bow_vectorizer.transform(new_doc)
new_doc_features = new_doc_features.todense()
print(new_doc_features)

[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [5]:
feature_names = bow_vectorizer.get_feature_names()
print(feature_names)

['and', 'and sky', 'and sky is', 'beautiful', 'beautiful sky', 'beautiful sky is', 'blue', 'blue and', 'blue and sky', 'blue cheese', 'cheese', 'is', 'is beautiful', 'is blue', 'is blue and', 'is so', 'is so blue', 'love', 'love blue', 'love blue cheese', 'sky', 'sky is', 'sky is beautiful', 'sky is blue', 'sky is so', 'so', 'so blue', 'the', 'the beautiful', 'the beautiful sky', 'the sky', 'the sky is']


上述输出显示出每个文档如何转换为向量，每行代表语料库中的一个文档。我们对两个语料库均执行相同操作。使用CORPUS变量中的文档建立了向量生成器。用它来提取特征，使用这个建立的向量生成器从全新的文档中提取特征。响亮的每一列描述的单词在feature_names变量中描述，每列的值是该词在文档中的频率，第一次看到他时可能难于理解，因此我们准备了以下函数，它有助于更好的理解特征向量。

In [6]:
import pandas as pd
def display_features(features,feature_names):
    df = pd.DataFrame(data = features,columns = feature_names)
    print(df)

In [7]:
display_features(features,feature_names)

   and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  blue  \
0    0        0           0          0              0                 0     1   
1    1        1           1          1              0                 0     1   
2    0        0           0          1              1                 1     1   
3    0        0           0          0              0                 0     1   

   blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0         0             0            0     ...                     0   
1         1             1            0     ...                     1   
2         0             0            0     ...                     0   
3         0             0            1     ...                     0   

   sky is blue  sky is so  so  so blue  the  the beautiful  the beautiful sky  \
0            1          0   0        0    1              0                  0   
1            1          0   0        0    0              0             

In [8]:
display_features(new_doc_features,feature_names)

   and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  blue  \
0    0        0           0          0              0                 0     1   

   blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0         0             0            0     ...                     0   

   sky is blue  sky is so  so  so blue  the  the beautiful  the beautiful sky  \
0            0          0   0        0    0              0                  0   

   the sky  the sky is  
0        0           0  

[1 rows x 32 columns]


### TD-IDF模型

词袋模型的向量依赖于单词出现的绝对频率。这存在一些潜在问题，语料库全部文档中出现次数较多的单词将会拥有较高的频率，这些词将会影响其他一些出现不如这些词频繁但对于文档分类更有意义和有效的单词。这就是TF-IDF的来源。TF-IDF代表的是词频—逆文档频率，是两个度量的组合。
数学上。TF-IDF是两个度量的乘机，可以表示为tfidf = tf*idf。
词频tf，由词袋模型计算得出。
逆文档词频由idf表示，是每个单词的文档频率的逆，该值由语料库中全部文档数量除以每个单词的文档频率，然后对结果应用对数运算变换其比例。在这里的实现中将对每个单词的文档频率+1，意味着词汇表中每个单词至少包含一个在语料库文档中，这是为了避免被0除的错误，平滑逆文档频率。也对idf的计算结果+1，避免被忽略单词拥有0值的idf。

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

def tfidf_transfromer(bow_matrix):
    transformer = TfidfTransformer(norm = 'l2',
                                   smooth_idf = True,
                                   use_idf = True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer,tfidf_matrix

在参数中使用了L2范数选项，并且对一些单词可能存在idf为0的情况以增加权重的方式对idf进行平滑处理，而没有忽略他们。在下面的代码片段观察这个函数的执行情况。

In [15]:
import numpy as np
#import tfidf_transfromer
feature_names = bow_vectorizer.get_feature_names()

tfidf_trans,tfidf_features = tfidf_transfromer(bow_features)
features = np.round(tfidf_features.todense(),2)
display_features(features,feature_names)

    and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  \
0  0.00     0.00        0.00       0.00           0.00              0.00   
1  0.26     0.26        0.26       0.20           0.00              0.00   
2  0.00     0.00        0.00       0.23           0.29              0.29   
3  0.00     0.00        0.00       0.00           0.00              0.00   

   blue  blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0  0.23      0.00          0.00         0.00     ...                  0.00   
1  0.13      0.26          0.26         0.00     ...                  0.26   
2  0.15      0.00          0.00         0.00     ...                  0.00   
3  0.23      0.00          0.00         0.44     ...                  0.00   

   sky is blue  sky is so    so  so blue   the  the beautiful  \
0         0.34       0.00  0.00     0.00  0.34           0.00   
1         0.20       0.00  0.00     0.00  0.00           0.00   
2         0.00       0.29  0.29  

现在我们看看该类内部如何工作。我们将载入必要的依赖，并通过重用词袋模型特征计算样例语料的单词频率(TF),该词频也可作为训练语料CORPUS的词频。

In [16]:
import scipy.sparse as sp
from numpy.linalg import norm
feature_names = bow_vectorizer.get_feature_names()

tf = bow_features.todense()
tf = np.array(tf,dtype='float64')

display_features(tf,feature_names)

   and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  blue  \
0  0.0      0.0         0.0        0.0            0.0               0.0   1.0   
1  1.0      1.0         1.0        1.0            0.0               0.0   1.0   
2  0.0      0.0         0.0        1.0            1.0               1.0   1.0   
3  0.0      0.0         0.0        0.0            0.0               0.0   1.0   

   blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0       0.0           0.0          0.0     ...                   0.0   
1       1.0           1.0          0.0     ...                   1.0   
2       0.0           0.0          0.0     ...                   0.0   
3       0.0           0.0          1.0     ...                   0.0   

   sky is blue  sky is so   so  so blue  the  the beautiful  \
0          1.0        0.0  0.0      0.0  1.0            0.0   
1          1.0        0.0  0.0      0.0  0.0            0.0   
2          0.0        1.0  1.0      1.0  1.0

我们将基于出现某单词的文档数量计算每个单词的文档频率(DF)下面的代码片段展示如何从词袋模型特征矩阵获得DF。

In [17]:
df = np.diff(sp.csc_matrix(bow_features,copy = True).indptr)
df = 1 + df
display_features([df],feature_names)

   and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  blue  \
0    2        2           2          3              2                 2     5   

   blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0         2             2            2     ...                     2   

   sky is blue  sky is so  so  so blue  the  the beautiful  the beautiful sky  \
0            3          2   2        2    3              2                  2   

   the sky  the sky is  
0        2           2  

[1 rows x 32 columns]


In [18]:
total_docs = 1 + len(CORPUS)
idf = 1.0 + np.log(float(total_docs) / df)
display_features([np.round(idf,2)],feature_names)

    and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  \
0  1.92     1.92        1.92       1.51           1.92              1.92   

   blue  blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0   1.0      1.92          1.92         1.92     ...                  1.92   

   sky is blue  sky is so    so  so blue   the  the beautiful  \
0         1.51       1.92  1.92     1.92  1.51           1.92   

   the beautiful sky  the sky  the sky is  
0               1.92     1.92        1.92  

[1 rows x 32 columns]


In [20]:
total_features = bow_features.shape[1]
idf_diag = sp.spdiags(idf,diags = 0,m = total_features,n = total_features)
idf = idf_diag.todense()
print(np.round(idf,2))

[[1.92 0.   0.   ... 0.   0.   0.  ]
 [0.   1.92 0.   ... 0.   0.   0.  ]
 [0.   0.   1.92 ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 1.92 0.   0.  ]
 [0.   0.   0.   ... 0.   1.92 0.  ]
 [0.   0.   0.   ... 0.   0.   1.92]]


In [23]:
tfidf = tf * idf
display_features(np.round(tfidf,2),feature_names)

    and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  \
0  0.00     0.00        0.00       0.00           0.00              0.00   
1  1.92     1.92        1.92       1.51           0.00              0.00   
2  0.00     0.00        0.00       1.51           1.92              1.92   
3  0.00     0.00        0.00       0.00           0.00              0.00   

   blue  blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0   1.0      0.00          0.00         0.00     ...                  0.00   
1   1.0      1.92          1.92         0.00     ...                  1.92   
2   1.0      0.00          0.00         0.00     ...                  0.00   
3   1.0      0.00          0.00         1.92     ...                  0.00   

   sky is blue  sky is so    so  so blue   the  the beautiful  \
0         1.51       0.00  0.00     0.00  1.51           0.00   
1         1.51       0.00  0.00     0.00  0.00           0.00   
2         0.00       1.92  1.92  

In [24]:
norms = norm(tfidf,axis = 1)
print(np.round(norms,2))

[4.44 7.43 6.57 4.4 ]


In [26]:
norm_tfidf = tfidf / norms[:,None]
display_features(np.round(norm_tfidf,2),feature_names)

    and  and sky  and sky is  beautiful  beautiful sky  beautiful sky is  \
0  0.00     0.00        0.00       0.00           0.00              0.00   
1  0.26     0.26        0.26       0.20           0.00              0.00   
2  0.00     0.00        0.00       0.23           0.29              0.29   
3  0.00     0.00        0.00       0.00           0.00              0.00   

   blue  blue and  blue and sky  blue cheese     ...      sky is beautiful  \
0  0.23      0.00          0.00         0.00     ...                  0.00   
1  0.13      0.26          0.26         0.00     ...                  0.26   
2  0.15      0.00          0.00         0.00     ...                  0.00   
3  0.23      0.00          0.00         0.44     ...                  0.00   

   sky is blue  sky is so    so  so blue   the  the beautiful  \
0         0.34       0.00  0.00     0.00  0.34           0.00   
1         0.20       0.00  0.00     0.00  0.00           0.00   
2         0.00       0.29  0.29  

### 高级词向量模型

为从文档中提取特征，有各种方法创建更高级的词向量模型。我们学习word2vec模型。

在代码实现中，将使用gensim库，该库是word2vecd的Python实现，提供几个高级的接口使得建模非常容易。基本思想是`提供一些文档语料作为输入，得到词向量特征作为输出`

在训练语料建立模型时，我们将主要关注下面的参数。
* size ：该参数用于设定词向量的维度，可以是几十到几千。可以尝试不同的维度，以获得最好的效果。
* window ：该参数用于设定语境或窗口尺寸，指定了训练时对算法来说可算作上下文的单词窗口长度。
* min_count ：该参数指定单词表中单词在语料中出现的最小次数。这个参数有助于去除一些文档中出现次数较少的不重要单词。
* simple ：该参数用于对单词出现的频率进行下采样，其理想值在0.01到0.0001之间。

建立模型后，将给予一些加权策略来定义和实现两种词向量与文档结合的技术。接下来将实现下面两个技术：
* 平均词向量
* TF-IDF加权词向量

In [28]:
import gensim
import nltk
TOKENIZED_CORPUS = [nltk.word_tokenize(sentence)
                   for sentence in CORPUS]
tokenized_new_doc = [nltk.word_tokenize(sentence)
                    for sentence in new_doc]
model = gensim.models.Word2Vec(TOKENIZED_CORPUS,size = 10,window = 10, min_count = 2,sample = 1e-3)

我们使用前面描述的参数建立模型。

#### 平均词向量

上面的模型为单词表中每个单词创建一个向量表示。

In [29]:
print(model['sky'])

[ 0.00743204 -0.02763226 -0.03154635 -0.02017373 -0.04970231  0.04648085
  0.0027055  -0.00576961 -0.00655493 -0.04238968]


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [30]:
print(model['blue'])

[ 0.04604309 -0.00202763  0.00298541  0.00483249  0.01052081  0.00169349
  0.04321021  0.02457611 -0.00708703 -0.01007829]


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


基于前面指定的参数尺寸，每个单词向量的长度为10。但是当我们处理语句和文档时，必须执行一些合并和聚合操作，以确保无论文本长度、单词数量等情况如何，最后特征的维度是相同的，在这个技术中，我们使用平均加权词向量，对于每个文档我们将提取文档中所有的单词，获得单词表中每个单词的词向量。我们将全部词向量加在一起，除以单词表中匹配单词的总数，最后得到文档的平均词向量表示结果。